# VISNLP Dataset

In [1]:
from torch.utils.data import Dataset,DataLoader
import joblib
import numpy as np
import cv2
from PIL import Image
import os
from torch import tensor
from torchvision import transforms
import matplotlib.pyplot as plt
import torch
from warnings import filterwarnings
filterwarnings('ignore')

class VisualNLPCustomDataset(Dataset):
    def __init__(self,dataframe,image_path):
        self.data = joblib.load(dataframe)
        self.image_path = image_path

    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        row = self.data.iloc[index]
        image_path = os.path.join(self.image_path, row['ImagePath'], row['Filename'])
        image = Image.open(image_path).convert('RGB')
        transform = transforms.ToTensor()
        image_tensor = transform(image)
        
        return image_tensor, row['Caption']
    
train = VisualNLPCustomDataset('../Dataset/Train_Data.pkl','../Data/Training/01.원천데이터')
image_tensor, text = train[0] # 3,512,512 (정규화된 Tensor Image: 소아 복부 X_ray : RGB), Caption: 'This plain abdominal radiograph reveals marked..'
text

'This plain abdominal radiograph of an about 1-month-old male infant shows markedly distended stomach.'

# VISUAL+NLP DataLoader

In [ ]:
train_dataloader = DataLoader(train,1,False)
batch_image_tensor, batch_nlp = next(iter(train_dataloader))
print(batch_image_tensor.shape) # (batchsize:3, 3, 512, 512)
print(batch_nlp) # (batchsize:3, 3개의 Texts)

torch.Size([1, 3, 512, 512])
('This plain abdominal radiograph of an about 1-month-old male infant shows markedly distended stomach.',)


: 

# Cross Attention - VIS + NLP Integrated Vector 확인하기

In [ ]:
from Visnlp import ViTFeatureExtractor, BertFeatureExtractor, CrossAttention, VISNLPEXTRACTOR

ex = BertFeatureExtractor()
vex = ViTFeatureExtractor(512,512,patch_size=8,embed_dim=160,num_heads=4,depth=8,in_channels=3)
vex(batch_image_tensor)
# visnlpextractor = VISNLPEXTRACTOR(512,512,8,312,4,12)
# visnlpextractor(batch_image_tensor, batch_nlp)

patches shape: torch.Size([1, 4096, 160])
pos_embedding shape: torch.Size([1, 4096, 160])
